In [75]:
from openai import OpenAI
from dotenv import load_dotenv
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import time
import requests

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

pd.set_option('display.max_rows', 100)

In [78]:
df = pd.read_csv('data/fetchGPT.csv')

# Get 50 entries each from Costco and Whole Foods Market
costco_df = df[df['SAMPLE_STORE'] == 'COSTCO'].head(200)
wholefoods_df = df[df['SAMPLE_STORE'] == 'WHOLE FOODS MARKET'].head(200)

# Combine the two dataframes
temp = pd.concat([costco_df, wholefoods_df], ignore_index=True)

# Verify the distribution
# print("Store distribution:")
# print(temp['SAMPLE_STORE'].value_counts())
temp

,ORIGINAL_ITEM_TEXT,FIDO_TYPE,DESCRIPTION,TIER1_NAME,BRAND,FIDO,PRODUCT_NUMBER,SAMPLE_STORE,SAMPLE_RECEIPT,ITEM_COUNT,MEDIAN(PAM.FINAL_PRICE),GMV
0,strawberries,PRODUCT,Strawberries,Produce,UNBRANDED,d18956ad-4eaa-408d-9620-26256eed1e4f,27003,COSTCO,6984257f-ae79-417b-9407-9d7fa9135954,348691,4.49,1565622.59
1,***kswtr40pk,PRODUCT,Kirkland Signature Purified Drinking Water,Beverages,KIRKLAND SIGNATURE,97f8b18c-640d-4105-9a2f-97f726996c5b,782796,COSTCO,14094048-0423-4e8e-8fa1-27f78b5823f3,283476,3.99,1131069.24
2,rotisserie,PRODUCT,Kirkland Signature Rotisserie Chicken,Deli & Bakery,KIRKLAND SIGNATURE,175c65d7-e9ce-4108-838c-8148098892ec,87745,COSTCO,dd890074-a2d9-4aaa-9f1b-9e5c48717e01,258676,4.99,1290793.24
3,ks cage free,PRODUCT,Kirkland Signature Large Eggs-cage Free-2 Dozen,Dairy,KIRKLAND SIGNATURE,12a677be-8339-42cc-9c15-495d6a80c0c0,637598,COSTCO,e2053422-40ba-4bb4-afc0-be69182020f2,189662,6.89,1306771.18
4,18 ct eggs,PRODUCT,Sauders Eggs Large White 18 Ct,Dairy,SAUDER,5875bf31-3674-4ceb-a4cc-437bc46ec250,1008,COSTCO,17d39fed-4a3a-463b-82b6-eac9df1823d1,162319,5.19,842435.61
...,...,...,...,...,...,...,...,...,...,...,...,...
395,yogi og grn antx tea,PRODUCT,Yogi Green Tea Super Antioxidant - 16 Count - 1.12 Oz,Beverages,YOGI,acaab811-5696-4437-a1a7-4917a3eb07e6,NaN,WHOLE FOODS MARKET,ee8d5674-24e8-4672-bfac-4ef49fc67f6f,39,4.79,186.81
396,casc og ccoa crspy rc crl,PRODUCT,Cascadian Farm Organic Cocoa Crispy Rice Cereal - 12 Oz,Pantry,CASCADIAN FARM,8dd8ed2c-c97a-4f6a-bd3d-c04b91089b2c,NaN,WHOLE FOODS MARKET,dccaef62-a4df-47d7-a6e8-add78f90ce9f,39,4.55,177.45
397,glac focus vtwtr 20fz,PRODUCT,Vitaminwater Focus Kiwi Strawberry Flavored Water - 20 Oz,Beverages,VITAMINWATER,b4848954-a61d-4fe8-89b7-90229a0745ff,NaN,WHOLE FOODS MARKET,7171467f-5ea0-470b-af34-663a745de104,38,2.19,83.22
398,yogi og purely tea,PRODUCT,Yogi Tea Purely Peppermint Caffeine Free Tea - 16 Bags,Beverages,YOGI,6fdb80cc-26f0-4c1e-bd5f-00575adfd13b,NaN,WHOLE FOODS MARKET,0fcdf8a9-1339-49ea-801f-a1a815fab21d,37,4.99,184.63


In [ ]:

# --- Assumptions: ---
# 1. `temp` is your pandas DataFrame with columns:
#      - 'ORIGINAL_ITEM_TEXT'
#      - 'SAMPLE_STORE'
#      - optionally 'PRODUCT_NUMBER'
# 2. `client` is already initialized and authenticated, e.g.:
#      client = OpenAI(api_key="YOUR_KEY")

results = []

for idx, row in temp.iterrows():
    original_item_text = row['ORIGINAL_ITEM_TEXT']
    sample_store      = row['SAMPLE_STORE']
    product_mnumber    = row.get('PRODUCT_NUMBER', 'N/A')
    median_price      = row.get('MEDIAN(PAM.FINAL_PRICE)', 'N/A')
    
    prompt = f'''
    # Role and Objective
    You are an AI product description analyzer tasked with standardizing and expanding abbreviated product descriptions into clear, structured data. Your goal is to identify brands, categories, and expand abbreviated text while maintaining accuracy.

    # Instructions
    - Analyze the given abbreviated product description
    - Expand abbreviations without adding interpretive content
    - Identify the most likely brand based on text and product number
    - Categorize the product based on expanded description
    - Provide confidence scores for brand and category predictions

    ## Sub-categories for more detailed instructions
    - Expand what is directly implied in the text (e.g., "gl" to "glass", "bl" to "bottle"), unless there are specific annotations like counts or sizes that can be verified through web search, or brand.
    - Assign confidence scores from high/medium/low based on clarity of information
    - Consider store context when determining brand and category

    # Reasoning Steps
    1. Expand Abbreviations
       - Identify common product abbreviations
       - Convert to standard product terminology
       - Maintain original meaning without interpretation
       - The format should be: brand name, product type, size, packaging, provided that this information is available through web search. 

    2. Brand Analysis
       - Look for brand indicators in text
       - Consider store-specific context
       - Assess confidence in brand identification

    3. Category Assignment
       - Analyze product characteristics
       - Determine product type
       - Assign confidence based on clarity

    # Output Format
    JSON structure with:
    {{
        "brand": "Predicted brand",
        "brand_score": "Confidence score (high/medium/low)",
        "category": "Predicted category",
        "category_score": "Confidence score (high/medium/low)",
        "expanded_description": "Expanded product description",
        "reasoning": "Reasoning for predictions and description"
    }}

    # Examples
    ## Example 1
    Input: "campari 12oz gl bl"
    Output: {{
        "brand": "Campari",
        "brand_score": high,
        "category": "Spirits",
        "category_score": high,
        "expanded_description": "Campari 12oz Glass Bottle",
        "reasoning": "Clear brand name 'Campari' present. Common abbreviations 'gl bl' clearly indicate glass bottle. Spirit category evident from brand."
    }}

    ## Example 2
    Input: "18 ct eggs"
    Output: {{
        "brand": "Sauders",
        "brand_score": low,
        "category": "Eggs",
        "category_score": high,
        "expanded_description": "Sauders Eggs Large White 18 Ct	",
        "reasoning": "Product number 1008 at retailer COSTCO leads to Sauders Large Eggs."
    }}

    # Context
    Here is the product information:
    - Original Item Text: "{original_item_text}"
    - Store: "{sample_store}"
    - Product Number: "{product_number}" (if applicable)
    - Median Price: "${median_price}" (if applicable)
    
    # Additional Considerations
    - Use price as a signal for brand tier (premium/standard/value)
    - Consider price ranges typical for the category
    - Factor price into confidence scores when relevant

    # Final instructions and prompt to think step by step
    1. First, expand only the abbreviated terms in the original text
    2. Then identify brand based on expanded text
    3. Finally, categorize the product based on the complete information
    4. Provide clear reasoning for each decision
    5. Return structured JSON with confidence scores
    '''
    
    # Common Abbreviations and Terms
    # [Reserved for future implementation]

    # Store-Specific Categories
    # [Reserved for future implementation]

    # Product Naming Conventions
    # [Reserved for future implementation]


    # Send the prompt to the GPT model
    response = client.responses.create(
        model="gpt-4.1",
        tools=[{"type": "web_search_preview"}],
        input=prompt, 
        temperature=0.0
    )
    
    # Extract the JSON payload from the response
    output_text = response.output_text
    start = output_text.find('{')
    end   = output_text.rfind('}') + 1
    
    if start == -1 or end <= start:
        print(f"[Row {idx}] No JSON found. Output preview:\n{output_text[:200]}...\n")
        continue
    
    json_text = output_text[start:end]
    try:
        parsed = json.loads(json_text)
    except json.JSONDecodeError as e:
        print(f"[Row {idx}] JSON parsing error: {e}\nPayload:\n{json_text}\n")
        continue
    
    # Limit reasoning to first 50 words
    reasoning_full = parsed.get("reasoning", "")
    reasoning_words = reasoning_full.split()
    reasoning_snip  = " ".join(reasoning_words[:50])
    
    results.append({
        "ORIGINAL_ITEM_TEXT":   original_item_text,
        "SAMPLE_STORE":         sample_store,
        "PRODUCT_NUMBER":       product_number,
        "BRAND":                parsed.get("brand", "N/A"),
        "BRAND_SCORE":          parsed.get("brand_score", "N/A"),
        "CATEGORY":             parsed.get("category", "N/A"),
        "CATEGORY_SCORE":       parsed.get("category_score", "N/A"),
        "EXPANDED_DESCRIPTION": parsed.get("expanded_description", "N/A"),
        "REASONING_SNIPPET":    reasoning_snip
    })
    
    # Throttle requests to avoid rate limits
    time.sleep(0.2)

# Convert to DataFrame
results_df = pd.DataFrame(results)

# (Optional) Save to CSV
# results_df.to_csv('data/fetchGPT_results.csv', index=False)

# Show the first few rows
results_df.head()

,ORIGINAL_ITEM_TEXT,SAMPLE_STORE,PRODUCT_NUMBER,BRAND,BRAND_SCORE,CATEGORY,CATEGORY_SCORE,EXPANDED_DESCRIPTION,REASONING_SNIPPET
0,strawberries,COSTCO,27003,Unspecified,low,Fresh Produce,high,"Premium Strawberries, 2 lb clamshell","The product description 'strawberries' corresponds to Costco's item number 27003, which is listed as 'Premium Strawberries' in a 2 lb clamshell. The brand is not specified in the available information, leading to a low confidence score for brand identification. However, the product clearly falls under the 'Fresh Produce' category, resulting"
1,***kswtr40pk,COSTCO,782796,Kirkland Signature,high,Bottled Water,high,"Kirkland Signature Purified Drinking Water, 16.9 fl oz, 40-pack","The original item text '***kswtr40pk' likely abbreviates 'Kirkland Signature Water 40 Pack.' The product number '782796' corresponds to Kirkland Signature Purified Drinking Water, 16.9 fl oz, 40-pack, confirming the brand and product details. The category is identified as 'Bottled Water' based on the product description."
2,rotisserie,COSTCO,87745,Kirkland Signature,high,Prepared Foods,high,Kirkland Signature Rotisserie Chicken,"The term 'rotisserie' refers to a cooking method involving roasting meat on a rotating spit. At Costco, the 'Kirkland Signature Rotisserie Chicken' is a well-known product, identified by item number 87745. This product is consistently priced at $4.99 and is recognized for its quality and value. Given the store context"
3,ks cage free,COSTCO,637598,Kirkland Signature,high,Eggs,high,"Kirkland Signature Cage-Free Large Eggs, 24 Count","The original item text 'ks cage free' likely stands for 'Kirkland Signature cage-free.' The product number 637598 corresponds to Kirkland Signature Large Cage-Free Eggs, 2 Dozen, as confirmed by Costco's product listing. Therefore, the brand is Kirkland Signature, and the product falls under the 'Eggs' category."
4,18 ct eggs,COSTCO,1008,Kirkland Signature,high,Eggs,high,"Kirkland Signature Large Grade A Eggs, 18 Count","The product description '18 ct eggs' refers to an 18-count package of eggs. At Costco, item number 1008 corresponds to 'Large Eggs, Grade A, 18 ct' ([costcobusinessdelivery.com](https://www.costcobusinessdelivery.com/large-eggs%2C-grade-a%2C-18-ct.product.2001138745.html?utm_source=openai)). While the specific brand isn't explicitly stated in the available information, Costco frequently sells eggs under its Kirkland Signature brand. Given this context,"


In [ ]:
# Role and Objective
# Instructions
## Sub-categories for more detailed instructions
# Reasoning Steps
# Output Format
# Examples
## Example 1
# Context
# Final instructions and prompt to think step by step



In [80]:
# Merge the results_df with temp DataFrame
merged_df = pd.merge(
    temp, 
    results_df,
    on=['ORIGINAL_ITEM_TEXT', 'SAMPLE_STORE', 'PRODUCT_NUMBER'],
    how='left'
)

# Display the merged results
merged_df

# Optionally save to CSV
merged_df.to_csv('data/merged_results.csv', index=False)